In [5]:
import string
import pandas as pd
import nltk
nltk.download('stopwords',quiet=True)
from nltk.corpus import stopwords
nltk.download('punkt',quiet=True)
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud

In [6]:
#import data
df_train=pd.read_csv('twittertrain.csv')
df_test=pd.read_csv('twittertest.csv')
df_sample=pd.read_csv('sample_submission.csv')

In [7]:
df_test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
df_train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
df_sample.head(5)

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [10]:
df_train['keyword']=df_train['keyword'].fillna('')

In [11]:
long_string=' '.join(df_train['keyword'])

In [12]:
#Preprocessing
df_train.head(5)

,id,keyword,location,text,target
0,1,,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,,NaN,All residents asked to 'shelter in place' are ...,1
3,6,,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
# Remove Punctuation
punc_list = list(string.punctuation)
def remove_punctuation(text, punct_list):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text

In [14]:
df_train2=df_train.copy()
df_train2['text']=df_train2['text'].apply(lambda x:remove_punctuation(x,punc_list))
display(df_train2.head(5))
df_test2=df_test.copy()
df_test2['text']=df_test2['text'].apply(lambda x:remove_punctuation(x,punc_list))
display(df_test2.head(5))

,id,keyword,location,text,target
0,1,,NaN,Our Deeds are the Reason of this earthquake M...,1
1,4,,NaN,Forest fire near La Ronge Sask Canada,1
2,5,,NaN,All residents asked to shelter in place are ...,1
3,6,,NaN,13 000 people receive wildfires evacuation or...,1
4,7,,NaN,Just got sent this photo from Ruby Alaska as ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,Heard about earthquake is different cities s...
2,3,NaN,NaN,there is a forest fire at spot pond geese are...
3,9,NaN,NaN,Apocalypse lighting Spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
# Tokenize each text column
token_df_test = df_test.copy()
for column in df_test.columns:
    token_df_test[column] = df_test[column].apply(lambda s: word_tokenize(s) if type(s) == str else s)
display(token_df_test.head(5))

,id,keyword,location,text
0,0,NaN,NaN,"[Just, happened, a, terrible, car, crash]"
1,2,NaN,NaN,"[Heard, about, #, earthquake, is, different, c..."
2,3,NaN,NaN,"[there, is, a, forest, fire, at, spot, pond, ,..."
3,9,NaN,NaN,"[Apocalypse, lighting, ., #, Spokane, #, wildf..."
4,11,NaN,NaN,"[Typhoon, Soudelor, kills, 28, in, China, and,..."


In [16]:
# Tokenize each text column
token_df_train = df_train2.copy()
for column in df_train2.columns:
    token_df_train[column] = df_train2[column].apply(lambda s: word_tokenize(s) if type(s) == str else s)
display(token_df_train.head(5))

,id,keyword,location,text,target
0,1,[],NaN,"[Our, Deeds, are, the, Reason, of, this, earth...",1
1,4,[],NaN,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,5,[],NaN,"[All, residents, asked, to, shelter, in, place...",1
3,6,[],NaN,"[13, 000, people, receive, wildfires, evacuati...",1
4,7,[],NaN,"[Just, got, sent, this, photo, from, Ruby, Ala...",1


In [17]:
import gensim

In [18]:
s1 = token_df_train['text']
s2 = token_df_test['text']

In [19]:
sentence = pd.concat([s1, s2], axis=0)

In [20]:
sentence.head(10)

0    [Our, Deeds, are, the, Reason, of, this, earth...
1        [Forest, fire, near, La, Ronge, Sask, Canada]
2    [All, residents, asked, to, shelter, in, place...
3    [13, 000, people, receive, wildfires, evacuati...
4    [Just, got, sent, this, photo, from, Ruby, Ala...
5    [RockyFire, Update, California, Hwy, 20, close...
6    [flood, disaster, Heavy, rain, causes, flash, ...
7    [I, m, on, top, of, the, hill, and, I, can, se...
8    [There, s, an, emergency, evacuation, happenin...
9    [I, m, afraid, that, the, tornado, is, coming,...
Name: text, dtype: object

In [21]:
print(len(sentence))

10876


In [22]:
print(type(sentence))
print(sentence.shape)
print(sentence)

<class 'pandas.core.series.Series'>
(10876,)
0       [Our, Deeds, are, the, Reason, of, this, earth...
1           [Forest, fire, near, La, Ronge, Sask, Canada]
2       [All, residents, asked, to, shelter, in, place...
3       [13, 000, people, receive, wildfires, evacuati...
4       [Just, got, sent, this, photo, from, Ruby, Ala...
                              ...                        
3258    [EARTHQUAKE, SAFETY, LOS, ANGELES, ÛÒ, SAFETY...
3259    [Storm, in, RI, worse, than, last, hurricane, ...
3260    [Green, Line, derailment, in, Chicago, http, :...
3261    [MEG, issues, Hazardous, Weather, Outlook, (, ...
3262    [#, CityofCalgary, has, activated, its, Munici...
Name: text, Length: 10876, dtype: object


In [23]:
sentence.iloc[1]

['Forest', 'fire', 'near', 'La', 'Ronge', 'Sask', 'Canada']

In [24]:
'Forest' in sentence.iloc[1]

True

In [25]:
w2v_model = gensim.models.Word2Vec(sentence,min_count=1,vector_size=100)

In [26]:
print(w2v_model.wv['forest'])

[-0.25549695  0.09498603  0.10216714  0.11509372  0.08555015 -0.31122643
  0.10691178  0.51979774 -0.1158135  -0.23377244  0.04678385 -0.35436544
  0.10528195  0.28809258  0.02181929 -0.12084446  0.14027652 -0.3211992
  0.13843831 -0.38372025  0.11493442  0.20492817  0.08681488 -0.32749006
  0.13416561  0.04210646 -0.16969758 -0.0992781  -0.15694453  0.12770875
  0.37820265 -0.10175475  0.01320067 -0.14098212 -0.21761863  0.3670432
  0.06304669 -0.23274088 -0.21112385 -0.26825777  0.10722826 -0.23184825
 -0.16697626  0.0124242   0.24779549 -0.05453163 -0.31712726 -0.06259139
  0.08015624  0.03271964 -0.03275778 -0.13351391 -0.08331414  0.0726935
 -0.10989518  0.16605598  0.1920443   0.09491955 -0.12463906  0.02024891
  0.07140468  0.04887406  0.09023403 -0.00894317 -0.11897476  0.3656171
 -0.1844727   0.03205688 -0.23070969  0.11921398 -0.03944146  0.17956474
  0.21690723  0.02823769  0.17833075 -0.16753055  0.02922731 -0.10259869
 -0.0816557   0.06980731 -0.18765017 -0.0149525  -0.195

In [27]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [28]:
most_similar(w2v_model,['our','title','client','company'])

,our,cos,title,cos,client,cos,company,cos
0,their,0.999787,ArianaGrande,0.945711,occurred,0.964432,due,0.996947
1,My,0.999765,@,0.945667,Being,0.962978,collapsed,0.996837
2,damage,0.999619,previous,0.945592,Mountain,0.962790,also,0.996788
3,cause,0.999592,NA,0.945570,Runs,0.962372,last,0.996748
4,new,0.999587,Amsterdam,0.945426,AUTOMATIC,0.962282,kids,0.996687
5,off,0.999541,Mad,0.945333,air,0.961733,an,0.996623
6,home,0.999540,HERE,0.945304,Galactic,0.961673,believe,0.996554
7,And,0.999477,recount,0.945179,resulted,0.961489,before,0.996554
8,gets,0.999463,Drive,0.944719,attacked,0.961455,man,0.996552
9,change,0.999458,charges,0.944614,attack,0.961426,too,0.996485


In [29]:
#Model Training
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score

In [31]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(token_df_train['text'])

x_train=tokenizer.texts_to_sequences(token_df_train['text'])

In [32]:
token_df_train['text'][0][:10]

['Our',
 'Deeds',
 'are',
 'the',
 'Reason',
 'of',
 'this',
 'earthquake',
 'May',
 'ALLAH']

In [33]:
x_train[0][:10]

[118, 4581, 26, 4, 862, 8, 22, 261, 138, 1612]

In [34]:
word_index=tokenizer.word_index
for word,num in word_index.items():
  print(f"{word}->{num}")
  if num==10:
    break

t->1
co->2
http->3
the->4
a->5
in->6
to->7
of->8
i->9
and->10


In [36]:
import numpy as np
nos=np.array([len(x) for x in x_train])
len(nos[nos<700])

7613

In [37]:
#Lets keep all description to 700, add padding to news with less than 700 words and truncating long on es
maxlen=700

#Making all description of size maxlen defiend aboe
x_train=pad_sequences(x_train,maxlen=maxlen)

In [38]:
#all description has 700 words(in numerical form now). If they had less words, they have been padded with 0
#0 is not associated to any word, as mapping of words started from 1
#0 will also be used later, if unknows word is encountered in test set
len(x_train[0])

700

In [39]:
#Adding 1 because of reserved 0 index
#Embedding layer creates one more vector for "UNKNOWN" words, or padded words(0s). This Vector is filled with zeros.
#Thus our vocab size increases by 1
vocab_size=len(tokenizer.word_index)+1

In [40]:
#Function to create weight matrix from word2vec genism model
def get_weight_matrix(model,vocab):
  #total vocabulary size plus 0 for unknown words
  vocab_size=len(vocab)+1
  #define weight matrix dimensions with all 0
  weight_matrix=np.zeros((vocab_size,EMBEDDING_DIM))
  #step vocab, store vectors using the Tokenizer's integer mapping
  for word,i in vocab.items():
    if word in model.wv:
      weight_matrix[i]=model.wv[word]
  return weight_matrix

In [41]:
print(type(word_index))
print(word_index['forest'])
print(w2v_model)

<class 'dict'>
195
Word2Vec<vocab=34450, vector_size=100, alpha=0.025>


In [42]:
EMBEDDING_DIM=100

In [43]:
#Getting embedding vectors from word2vec and using it as weights of non-trainable keras embedding layer
embedding_vectors=get_weight_matrix(w2v_model,word_index)

In [45]:
#Defining Neural Network
model=Sequential()
#Non-trainable embedding layer
model.add(Embedding(vocab_size,output_dim=EMBEDDING_DIM,weights=[embedding_vectors],input_length=maxlen,trainable=False))
#LSTM
model.add(LSTM(units=128))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

del embedding_vectors

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 700, 100)          2195100   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,312,477
Trainable params: 117,377
Non-trainable params: 2,195,100
_________________________________________________________________
